In [1]:
%run C:/Users/MohammedSB/Desktop/projects/Hypertension/Requirements.ipynb

In [2]:
# Seed
set_seed(0)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
PATH = r"D:\\MohammedSB\\Fundus\\"
CSV_PATH = {"HTNPath": PATH + r"HTN", "NonHTNPath": PATH + "NonHTN"}

MODELS_PATH = r"C:\Users\MohammedSB\Desktop\projects\Hypertension\models"

In [5]:
train_set, test_set = get_datasets()

In [6]:
BATCH_SIZE = 8

train_dataset = TabularDataset(train_set)
test_dataset = TabularDataset(test_set)

train_x, train_y = train_set[["Age", "Gender"]], train_set["Hypertension"]

train_loader, test_loader = train_test_loader(train_dataset, test_dataset, batch_size=BATCH_SIZE)

# val_dataset = TabularDataset(val_set)
# val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# XGBoost

In [7]:
boost = xgb.XGBClassifier(tree_method='gpu_hist', objective="binary:logistic")

In [8]:
param_grid = {'max_depth': [3,6,10],
           'learning_rate': [0.0001, 0.005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.3],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

clf = GridSearchCV(estimator=boost, 
                   param_grid=param_grid,
                   scoring='f1', 
                   verbose=1)

clf.fit(train_x, train_y)

print(clf.best_score_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.7640520063484061


[Parallel(n_jobs=1)]: Done 720 out of 720 | elapsed:  9.3min finished


In [9]:
with open(MODELS_PATH+'/DemographicXGBParams.json', 'w', encoding='utf-8') as f:
    json.dump(clf.best_params_, f, ensure_ascii=False, indent=4)

# SVM

In [10]:
svm = SVC()

In [11]:
param_grid = [
    {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
    {'C': [1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf']},
    {'C': [1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001], 'degree': [1,2,3], 'kernel': ['poly']}
]

clf = GridSearchCV(estimator=svm, 
                   param_grid=param_grid,
                   scoring='f1', 
                   verbose=1)

clf.fit(train_x, train_y)

print(clf.best_score_)

Fitting 5 folds for each of 124 candidates, totalling 620 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.7648120751942031


[Parallel(n_jobs=1)]: Done 620 out of 620 | elapsed: 100.7min finished


In [12]:
with open(MODELS_PATH+'/DemographicSVMParams.json', 'w', encoding='utf-8') as f:
    json.dump(clf.best_params_, f, ensure_ascii=False, indent=4)

# FCNN

In [20]:
tabular_model = nn.Sequential(
    nn.Linear(in_features=2, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=32),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=32, out_features=16),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=16, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=1),
)

In [21]:
tabular_model = tabular_model.to(device)

In [22]:
models = {
    "image_model": None,
    "tabular_model": tabular_model,
    "fusion_model": None
}

early_stop = {
    "patience": 20,
    "min_delta": 0,
    "multip": 10
}

epochs = 500
optimizer = torch.optim.Adam(tabular_model.to(device).parameters(), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()

In [23]:
metrics = train_val(epochs=epochs, models=models, train_loader=train_loader, val_loader=test_loader, 
                    criterion=criterion, optimizer=optimizer, device=device, early_stop=early_stop)

Batch in Progress:  12%|█████                                     | 14/115 [00:00<00:00, 123.36it/s]

Beginning Training: 

Epoch 1/500


Batch in Progress:   0%|                                                    | 0/115 [00:00<?, ?it/s]

Train Set:
    Average Loss:  0.695530,    Accuracy: 51.48%,    Correct Counter: 471/915,    F1 Score: 0.00,    Precision: 1.00,    Recall:  0.00 

Validation Set:
    Average Loss:  0.696637,    Accuracy: 50.44%,    Correct Counter: 115/228,    F1 Score: 0.00,    Precision: 1.00,    Recall:  0.00 

Epoch 2/500


Batch in Progress:   0%|                                                    | 0/115 [00:00<?, ?it/s]

...

Train Set:
    Average Loss:  0.532224,    Accuracy: 70.93%,    Correct Counter: 649/915,    F1 Score: 0.75,    Precision: 0.64,    Recall:  0.90 

Validation Set:
    Average Loss:  0.593727,    Accuracy: 68.42%,    Correct Counter: 156/228,    F1 Score: 0.74,    Precision: 0.63,    Recall:  0.89 

Epoch 39/500


Batch in Progress:   0%|                                                    | 0/115 [00:00<?, ?it/s]

Train Set:
    Average Loss:  0.532151,    Accuracy: 70.93%,    Correct Counter: 649/915,    F1 Score: 0.75,    Precision: 0.64,    Recall:  0.90 

Validation Set:
    Average Loss:  0.593822,    Accuracy: 68.42%,    Correct Counter: 156/228,    F1 Score: 0.74,    Precision: 0.63,    Recall:  0.89 

Epoch 40/500


Batch in Progress: 100%|███████████████████████████████████████████| 29/29 [00:00<00:00, 200.05it/s]

Train Set:
    Average Loss:  0.532082,    Accuracy: 70.93%,    Correct Counter: 649/915,    F1 Score: 0.75,    Precision: 0.64,    Recall:  0.90 

Validation Set:
    Average Loss:  0.593907,    Accuracy: 68.42%,    Correct Counter: 156/228,    F1 Score: 0.74,    Precision: 0.63,    Recall:  0.89 



In [25]:
torch.save(tabular_model.state_dict(), MODELS_PATH + r'\DemographicFCNN.pth')